In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt


class HeatDiffusion(object):
    def __init__(self, noise_steps=500, init_u=1000, alpha=2., delta=1e-4, N_terms=10, device="cuda"):
        self.device = device
        self.noise_steps = noise_steps
        
        self.init_u = init_u
        self.alpha = alpha
        self.delta = delta
        self.N_terms = N_terms
        
        self.N = int(2. / delta)
        
        self.vals = torch.linspace(0, 2, self.N, device=self.device)
        self.X, self.Y = torch.meshgrid([self.vals, self.vals])
        
    def forward_diffusion(self, x0, t):
        heat = torch.stack([self.compute_u(x, y, t, x0) for x in self.vals for y in self.vals]).reshape(self.N, self.N, -1).mean(dim=-1)
        threshold = 0.1 * heat.max()
        
        contour = plt.contour(self.X.cpu().numpy(), self.Y.cpu().numpy(), heat.cpu().numpy(), levels=[threshold])
        paths = contour.collections[0].get_paths()
        
        boundaries = []
        for path in paths:
            v = path.vertices
            boundaries.append(v)
            
        return boundaries

    def compute_A(self, n, m, x0):
        result = self.u0(self.X.unsqueeze(-1), self.Y.unsqueeze(-1), x0) * torch.sin(n * np.pi * self.X.unsqueeze(-1) / 2.) * torch.sin(m * np.pi * self.Y.unsqueeze(-1) / 2.) * self.delta**2
        integral = result.sum(dim=(0,1))
        return integral  
    
    def compute_u(self, x, y, t, x0):
        u = torch.zeros(x0.shape[0], device=self.device)
        for n in range(1, self.N_terms+1):
            for m in range(1, self.N_terms+1):
                A_nm = self.compute_A(n, m, x0)
                lambda_val = (n * np.pi / 2.)**2 + (m * np.pi / 2.)**2
                u += A_nm * torch.sin(n * np.pi * x / 2.) * torch.sin(m * np.pi * y / 2.) * torch.exp(-self.alpha * lambda_val * t)
        return u
    
    def u0(self, x, y, x0):
        x0_x, x0_y = x0[..., 0], x0[..., 1]
        mask = (x0_x <= x) & (x < x0_x+self.delta) & (x0_y <= y) & (y < x0_y+self.delta)
        return torch.where(mask, self.init_u / (self.delta**2), torch.tensor(0., device=self.device))
        
    def sample_timesteps(self, n):
        return torch.randint(1, self.noise_steps, (n,), device=self.device)



In [2]:
heat = HeatDiffusion()
x0 = torch.tensor([[[0.5, 0.5]]], dtype=torch.float, device='cuda')
posterior = heat.forward_diffusion(x0, 10)

/home/junwoo/anaconda3/envs/scorefield/lib/python3.8/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.49 GiB (GPU 0; 3.81 GiB total capacity; 2.98 GiB already allocated; 115.44 MiB free; 2.98 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
import numpy as np
print(np.array([1.]) < 1)

[False]
